In [114]:
import datetime
import numpy as np
from copy import deepcopy

In [115]:
class State:
  def __init__(self, start, pre_move = 'root'):
    self.RHS = start
    self.LHS = [0,0] #[#missionaries, #cannibals]
    self.goal = deepcopy(start)
    self.boad_location = 0 #0 is the RHS or the river, #1 is the LHS of the river
    self.pre_move = pre_move

  def __eq__(self, other):
    if isinstance(other, State):
      return self.RHS == other.RHS #and self.LHS == other.LHS and self.boad_location == other.boad_location
    
  def __hash__(self):
    return hash(tuple(self.RHS))

  def ValidMove(self, boad):
    if sum(boad) > 0 and sum(boad) <= 2:

      if self.boad_location == 0: #boad on RHS
        if self.RHS[0] - boad[0] >= 0 and self.RHS[1] - boad[1] >=0: #Prevent over substraction
          if self.RHS[0] - boad[0] == 0: #When one side no M after entering the boad
            if self.LHS[0] + boad[0] >= self.LHS[1] + boad[1]:
              return True
          elif self.LHS[0] + boad[0] == 0: #When one side dont have M after boad reach
            if self.RHS[0] - boad[0] >= self.RHS[1] - boad[1]:
              return True
          elif self.RHS[0] - boad[0] >= self.RHS[1] - boad[1] and self.LHS[0] + boad[0] >= self.LHS[1] + boad[1]:
            return True
          
      if self.boad_location == 1: #boad on LHS
        if self.LHS[0] - boad[0] >= 0 and self.LHS[1] - boad[1] >=0: #Prevent over substraction
          if self.LHS[0] - boad[0] == 0: #When one side no M after entering the boad
            if self.RHS[0] + boad[0] >= self.RHS[1] + boad[1]:
              return True
          elif self.RHS[0] + boad[0] == 0:
            if self.LHS[0] - boad[0] >= self.LHS[1] - boad[1]:
              return True
          elif self.LHS[0] - boad[0] >= self.LHS[1] - boad[1] and self.RHS[0] + boad[0] >= self.RHS[1] + boad[1]:
            return True
           
    return False

  def SavedMove(self, boad):
    saved = deepcopy(self)
    if self.ValidMove(boad):
      if self.boad_location == 0:
        self.RHS[0] -= boad[0]
        self.RHS[1] -= boad[1]
        self.LHS[0] += boad[0]
        self.LHS[1] += boad[1]
        self.pre_move = boad
        self.boad_location = 1
      elif self.boad_location == 1:
        self.RHS[0] += boad[0]
        self.RHS[1] += boad[1]
        self.LHS[0] -= boad[0]
        self.LHS[1] -= boad[1]
        self.pre_move = boad
        self.boad_location = 0
      return saved
    else:
      print('Invalid move')
      return False 

  def PossibleMoves(self):
    if self.pre_move == 'root':
      return [[2,0],[0,2],[1,1],[1,0],[0,1]]
    if self.pre_move == [2,0]:
      return [[0,2],[1,1],[1,0],[0,1]]
    elif self.pre_move == [0,2]:
      return [[2,0],[1,1],[1,0],[0,1]]
    elif self.pre_move == [1,1]:
      return [[2,0],[0,2],[1,0],[0,1]]
    elif self.pre_move == [1,0]:
      return [[2,0],[0,2],[1,1],[0,1]]
    elif self.pre_move == [0,1]:
      return [[2,0],[0,2],[1,1],[1,0]]

  def Expand(self):
    nodes = []
    for move in self.PossibleMoves():
      if self.ValidMove(move):
        temp = deepcopy(self)
        temp.SavedMove(move)
        nodes.append(temp)
    return nodes

    # boad_list = [[2,0],[0,2],[1,1],[1,0],[0,1]]
    # for i in range(0, len(boad_list)):
    #     if self.ValidMove(boad_list[i]) and self.pre_move != loop_move[i]:
    #       temp = deepcopy(self)
    #       temp.SavedMove(boad_list[i])
    #       nodes.append(temp)
    # return nodes

In [116]:
visited = {}
path = []
n_expand = 0

def DFS(state):
    global visited, path, n_expand
    if state in visited:
        return False
    visited[state] = True
    if state.LHS == state.goal:
        path.append(node.pre_move)
        return True
    else:
        for node in state.Expand():
            n_expand += 1
            found = DFS(node)
            if found:
                path.append(node.pre_move)
                return True
    del visited[state]
    return False


In [117]:
def ASearch(start):
    queue = [start]
    Inqueue = {start:True}
    visited = {start:True}
    history = {start:'root'}

In [118]:
s = [3,1]
s2= [0,2]
boad = 1
c = np.concatenate((s,s2))
c = np.append(c,boad)
print(c)

[3 1 0 2 1]


In [119]:
if __name__ == '__main__':
    start_state = [3,3] # First index = #Missionaries, Second index = #Cannibals
    start = State(start_state)
    found = DFS(start)
    print(found)
    path.reverse()
    print("Path found:", path)
    print('Number of moves:', len(path)-1)
    print('Number of nodes expaned: ', n_expand)
    
    # nodes = start.Expand()
    # temp = []
    # temp2 = []
    # temp3 = []
    # temp4 = []
    # for i in range(0, len(nodes)):
    #     print(i)
    #     print('RHS:', nodes[i].RHS)
    #     print('LHS: ', nodes[i].LHS)
    #     print('Previous move', nodes[i].pre_move)
    #     print('Goal', nodes[i].goal)
    #     print('Boad location: ', nodes[i].boad_location)
    #     print('----------------')
    #     temp.append(nodes[i])

    # state0 = temp[0]
    # state1 = temp[1]

    # nodes = state0.Expand()

    # for i in range(0, len(nodes)):
    #     print(i)
    #     print('RHS:', nodes[i].RHS)
    #     print('LHS: ', nodes[i].LHS)
    #     print('Previous move', nodes[i].pre_move)
    #     print('Goal', nodes[i].goal)
    #     print('Boad location: ', nodes[i].boad_location)
    #     print('----------------')
    #     temp2.append(nodes[i])
    
    # state0 = temp2[0]

    # nodes = state0.Expand()

    # for i in range(0, len(nodes)):
    #     print(i)
    #     print('RHS:', nodes[i].RHS)
    #     print('LHS: ', nodes[i].LHS)
    #     print('Previous move', nodes[i].pre_move)
    #     print('Goal', nodes[i].goal)
    #     print('Boad location: ', nodes[i].boad_location)
    #     print('----------------')
    #     temp3.append(nodes[i])

    # state0 = temp3[0]

    # nodes = state0.Expand()

    # for i in range(0, len(nodes)):
    #     print(i)
    #     print('RHS:', nodes[i].RHS)
    #     print('LHS: ', nodes[i].LHS)
    #     print('Previous move', nodes[i].pre_move)
    #     print('Goal', nodes[i].goal)
    #     print('Boad location: ', nodes[i].boad_location)
    #     print('----------------')
    #     temp4.append(nodes[i])


[3, 3]
[3, 1]
[3, 2]
[3, 0]
[2, 2]
[2, 2]
[3, 2]
[3, 0]
[3, 1]
[1, 1]
[3, 1]
[3, 2]
False
Path found: []
Number of moves: -1
Number of nodes expaned:  15
